In [103]:
# gen trigger

import os
import re
import json
import copy
import time
import random
import pickle

from utils.chat2DeepSeek import Chat2DeepSeek
from utils.gen_prompt import Prompt
from utils.util import parse_trigger_answer, parse_argument_answer, save_json, load_json

In [3]:
API_KEY = 'sk-553325d9958b4b3ba7bf15d297abb364'

In [5]:
event_dict = load_json('./meta_data/event_dict_full.json')

In [7]:
n_event = 3
n_trigger = 100
n_argument = 20
complex_score = [5, 10]
max_argument = 4


In [9]:
event_arg = {}
event_tri = {}

In [11]:
def get_local_time():
    return str(time.localtime().tm_mon) + '_' + str(time.localtime().tm_mday) + '_' + str(time.localtime().tm_hour) + '_' + str(time.localtime().tm_min)

In [13]:
def chat4argument_pool(gen_prompt, n_argument, event_type, event_def, role_dict):
    chat = Chat2DeepSeek(api_key=API_KEY)
    response = chat.prompt2chat(gen_prompt.gen_argument_promt(n_argument, event_type, event_def, role_dict)).strip()
    a_d = re.findall('```json(.*?)```', response, flags=re.S)[0]
    try:
        return json.loads(a_d)
    except:
        return a_d

In [15]:
def chat4trigger_pool(gen_prompt, n_trigger, event_type, event_def, example, exam_trigger):
    chat = Chat2DeepSeek(api_key=API_KEY)
    response = chat.prompt2chat(gen_prompt.gen_trigger_prompt(n_trigger, event_type, event_def, example, exam_trigger)).strip()
    a_d = re.findall('```json(.*?)```', response, flags=re.S)[0]
    try:
        return json.loads(a_d)
    except:
        return a_d

In [17]:
def chat4argument_pool_all(event_dict, n_argument):
    for i, event in enumerate(event_dict, 1):
        # event_dict = {'ATTACK':[event_def, role_dict, example, exam_trigger]}
        try:
            if event_arg.get(event):
                print('---EVENT {} chated'.format(event))
                continue
            gen_prompt = Prompt()
            event_type = event
            event_def = event_dict[event][0]
            role_dict = event_dict[event][1]
    
            start_time = time.time()
            print('TIME {} -- chating for EVENT {}'.format(get_local_time(), event))
            a_d = chat4argument_pool(gen_prompt, n_argument, event_type, event_def, role_dict)
            print('TIME {} -- done chating'.format(get_local_time()))
            if type(a_d) == type({}):
                event_arg[event] = a_d
            else:
                print('error ', event)
                try:
                    with open('./log/error/' + event + '_arg.txt', 'w') as f:
                        f.write(a_d)
                except:
                    print('---can not save:', a_d)
            end_time = time.time()
            print('Done processing EVENT: {}, in {} s'.format(event, end_time - start_time))
            time.sleep(5)
        except:
            continue
        save_json(event_arg, './arg_{}_{}_{}.json'.format(i, event, get_local_time()))
    print('ALL DONE')

In [19]:
def chat4trigger_pool_all(event_dict, n_trigger):
    
    for i, event in enumerate(event_dict, 1):
        # event_dict = {'ATTACK':[event_def, role_dict, example, exam_trigger, father_event]}
        try:
            if event_tri.get(event):
                print('---EVENT {} chated'.format(event))
                continue
            gen_prompt = Prompt()
            event_type = event
            event_def = event_dict[event][0]
            example = event_dict[event][2]
            exam_trigger = event_dict[event][3]
            
            start_time = time.time()
            print('TIME {} -- chating for EVENT {}'.format(get_local_time(), event))
            a_d = chat4trigger_pool(gen_prompt, n_trigger, event_type, event_def, example, exam_trigger)
            print('TIME {} -- done chating'.format(get_local_time()))
            if type(a_d) == type({}):
                event_tri[event] = a_d['triggers']
            else:
                print('error ', event)
                try:
                    with open('./log/error/' + event + '_trigger.txt', 'w') as f:
                        f.write(a_d)
                except:
                    print('---can not save:', a_d)
            end_time = time.time()
            print('Done processing EVENT: {}, in {} s'.format(event, end_time - start_time))
            time.sleep(5)
        except:
            continue
        save_json(event_tri, './tri_{}_{}_{}.json'.format(i, event, get_local_time()))
    print('ALL DONE')

In [42]:
chat4trigger_pool_all(event_dict, n_trigger)

---EVENT START-ORG chated
---EVENT MERGE-ORG chated
---EVENT DECLARE-BANKRUPTCY chated
---EVENT END-ORG chated
---EVENT ATTACK chated
TIME 3_3_1_44 -- chating for EVENT DEMONSTRATE
TIME 3_3_2_4 -- done chating
Done processing EVENT: DEMONSTRATE, in 1203.6810545921326 s
---EVENT MEET chated
---EVENT PHONE-WRITE chated
---EVENT ARREST-JAIL chated
TIME 3_3_2_4 -- chating for EVENT RELEASE-PAROLE
TIME 3_3_2_7 -- done chating
Done processing EVENT: RELEASE-PAROLE, in 212.9145691394806 s
---EVENT TRIAL-HEARING chated
---EVENT CHARGE-INDICT chated
---EVENT SUE chated
---EVENT CONVICT chated
---EVENT SENTENCE chated
---EVENT FINE chated
---EVENT EXECUTE chated
TIME 3_3_2_7 -- chating for EVENT EXTRADITE
TIME 3_3_2_9 -- done chating
Done processing EVENT: EXTRADITE, in 118.2546775341034 s
TIME 3_3_2_9 -- chating for EVENT ACQUIT
TIME 3_3_2_11 -- done chating
Done processing EVENT: ACQUIT, in 73.54216623306274 s
---EVENT APPEAL chated
TIME 3_3_2_11 -- chating for EVENT PARDON
TIME 3_3_2_13 -- do

In [ ]:
while len(event_arg) < 33:
    chat4argument_pool_all(event_dict, n_argument)
    print('--------------------------------------')

no orignal record
TIME 6_5_22_25 -- chating for EVENT START-ORG
TIME 6_5_22_29 -- done chating
Done processing EVENT: START-ORG, in 186.87589597702026 s
no orignal record
TIME 6_5_22_29 -- chating for EVENT MERGE-ORG
TIME 6_5_22_31 -- done chating
error  MERGE-ORG
---can not save: 
{
  "Org": {
    "Organization": [
      "company", "firm", "agency", "corporation", "enterprise", "institution", "association", "group", "alliance", "venture", 
      "partnership", "cooperative", "conglomerate", "syndicate", "trust", "union", "society", "authority", "ministry", "department",
      "bureau", "office", "division", "branch", "subsidiary", "affiliate", "cartel", "consortium", "guild", "league"
    ]
  },
  "Time": {
    "Time": [
      "now", "then", "today", "yesterday", "tomorrow", "soon", "later", "recently", "currently", "previously", 
      "afterwards", "formerly", "immediately", "moment", "period", "date", "time", "quarter", "season", "year",
      "month", "week", "decade", "century", 

In [43]:
len(event_tri)

33

In [45]:
# save_json(event_tri, './trigger_pool.json')

In [52]:
# save_json(event_arg, './arg_1_START-ORG_{}.json'.format(get_local_time()))

In [147]:
arg_fp = './meta_data/argument_pool_augmented_5_16.json'
arg_dic = load_json(arg_fp)

In [149]:
fa_arg_dic = {}
for e in arg_dic:
    fa_e = event_dict[e][-1]
    if fa_e not in fa_arg_dic:
        fa_arg_dic[fa_e] = {}
    fa_arg_dic[fa_e][e] = arg_dic[e]

In [151]:
def map_args(sub):
    m_arg_dic = {}
    for e in sub:
        for r in sub[e]:
            if r not in m_arg_dic:
                m_arg_dic[r] = []
            m_arg_dic[r].extend(sub[e][r])
            
    for e in sub:
        for r in sub[e]:
            if r in m_arg_dic:
                sub[e][r] = m_arg_dic[r]

In [153]:
for sub in fa_arg_dic:
    map_args(fa_arg_dic[sub])

In [155]:
map_arg_dic = {}
for e in fa_arg_dic:
    for sub_e in fa_arg_dic[e]:
        map_arg_dic[sub_e] = fa_arg_dic[e][sub_e]

In [157]:

s = []
for e in map_arg_dic:
    for r in map_arg_dic[e]:
        for a in map_arg_dic[e][r]:
            if len(a) <= 1:
                map_arg_dic[e][r].remove(a)
                s.append(a)

In [159]:
map_arg_dic.keys()

dict_keys(['TRANSPORT', 'ELECT', 'START-POSITION', 'NOMINATE', 'END-POSITION', 'ATTACK', 'DEMONSTRATE', 'MEET', 'PHONE-WRITE', 'MARRY', 'INJURE', 'DIE', 'BE-BORN', 'DIVORCE', 'TRANSFER-MONEY', 'TRANSFER-OWNERSHIP', 'SUE', 'ARREST-JAIL', 'EXECUTE', 'TRIAL-HEARING', 'SENTENCE', 'CHARGE-INDICT', 'CONVICT', 'RELEASE-PAROLE', 'FINE', 'PARDON', 'APPEAL', 'EXTRADITE', 'ACQUIT', 'END-ORG', 'START-ORG', 'DECLARE-BANKRUPTCY', 'MERGE-ORG'])

In [161]:
s

['B', 'B']

In [101]:
save_json(map_arg_dic, './meta_data/argument_pool_augmented_map_6_18.json')

In [89]:
for f_e in fa_arg_dic:
    print(f_e)
    for e in fa_arg_dic[f_e]:
        print('\t', e)
        for r in fa_arg_dic[f_e][e]:
            print('\t\t{}:{}'.format(r, len(fa_arg_dic[f_e][e][r])))

MOVEMENT
	 TRANSPORT
		Vehicle:40
		Artifact:151
		Destination:204
		Agent:44
		Origin:90
		Place:1
PERSONNEL
	 ELECT
		Person:200
		Entity:120
		Place:110
	 START-POSITION
		Person:200
		Entity:120
		Place:110
	 NOMINATE
		Person:200
		Agent:33
	 END-POSITION
		Entity:120
		Person:200
		Place:110
CONFLICT
	 ATTACK
		Place:196
		Target:182
		Attacker:183
		Instrument:70
		Victim:1
	 DEMONSTRATE
		Entity:35
		Place:196
CONTACT
	 MEET
		Entity:202
		Place:48
	 PHONE-WRITE
		Entity:202
		Place:48
LIFE
	 MARRY
		Person:100
		Place:190
	 INJURE
		Victim:176
		Agent:108
		Place:190
		Instrument:71
	 DIE
		Victim:176
		Agent:108
		Place:190
		Instrument:71
	 BE-BORN
		Place:190
		Person:100
	 DIVORCE
		Person:100
TRANSACTION
	 TRANSFER-MONEY
		Giver:58
		Recipient:55
		Beneficiary:64
		Place:73
	 TRANSFER-OWNERSHIP
		Buyer:42
		Artifact:38
		Seller:34
		Place:73
		Beneficiary:64
JUSTICE
	 SUE
		Plaintiff:53
		Defendant:249
		Adjudicator:292
		Place:378
	 ARREST-JAIL
		Person:111
		Agent:97
		

In [91]:
fa_arg_dic['PERSONNEL']['ELECT']['Person']

['man',
 'Franklin Roosevelt',
 'person',
 'speaker',
 'john edwards',
 'Schwarzenegger',
 'Dean',
 'guy',
 'Bush',
 'candidates',
 'John Edwards',
 'John Kerry',
 'Edwards',
 'Chiluba',
 'lawmaker',
 'candidate',
 'Kerry',
 'Sistani',
 'figure heads',
 'Lincoln',
 'Americans',
 'Douglas Wilder',
 'Mark Warner',
 'Chichester',
 'BUSH',
 'Mahmoud Abbas',
 'leaders',
 'leader',
 'thugs',
 'president',
 'nominee',
 'contestant',
 'governor',
 'winner',
 'chairperson',
 'senator',
 'commissioner',
 'professionals',
 'politicians',
 'Shah',
 'workers',
 'peter arnett',
 'gerald aftery',
 'leung',
 'jordan',
 'priest',
 'deacons',
 'people',
 'father',
 'charles',
 "sean patrick o'malley",
 'girl',
 'figures',
 'Bush',
 'Toefting',
 'Akio Toyoda',
 'Al - Douri',
 'Suharto',
 'Jay Garner',
 'Garner',
 'Ahmed Chalabi',
 'Barzan Ibrahim',
 'Paul Silas',
 'adviser',
 'Miroslav Kostelka',
 'member',
 'rich',
 'chief',
 'Condi Rice',
 "Ba'athists",
 'officers',
 'Republican Guards',
 'Mukhabarat',

In [99]:
for e in map_arg_dic:
    print(e)
    for r in map_arg_dic[e]:
        print('\t', r)

TRANSPORT
	 Vehicle
	 Artifact
	 Destination
	 Agent
	 Origin
	 Place
ELECT
	 Person
	 Entity
	 Place
START-POSITION
	 Person
	 Entity
	 Place
NOMINATE
	 Person
	 Agent
END-POSITION
	 Entity
	 Person
	 Place
ATTACK
	 Place
	 Target
	 Attacker
	 Instrument
	 Victim
DEMONSTRATE
	 Entity
	 Place
MEET
	 Entity
	 Place
PHONE-WRITE
	 Entity
	 Place
MARRY
	 Person
	 Place
INJURE
	 Victim
	 Agent
	 Place
	 Instrument
DIE
	 Victim
	 Agent
	 Place
	 Instrument
BE-BORN
	 Place
	 Person
DIVORCE
	 Person
TRANSFER-MONEY
	 Giver
	 Recipient
	 Beneficiary
	 Place
TRANSFER-OWNERSHIP
	 Buyer
	 Artifact
	 Seller
	 Place
	 Beneficiary
SUE
	 Plaintiff
	 Defendant
	 Adjudicator
	 Place
ARREST-JAIL
	 Person
	 Agent
	 Place
EXECUTE
	 Place
	 Agent
	 Person
TRIAL-HEARING
	 Defendant
	 Place
	 Adjudicator
	 Prosecutor
SENTENCE
	 Defendant
	 Adjudicator
	 Place
CHARGE-INDICT
	 Adjudicator
	 Defendant
	 Prosecutor
	 Place
CONVICT
	 Defendant
	 Adjudicator
	 Place
RELEASE-PAROLE
	 Entity
	 Person
	 Place
FINE
	 En